In [ ]:
# !pip install pymongo
from pymongo import MongoClient

# Assuming you have a MongoClient instance 'client' connected to your MongoDB server
client = MongoClient('<mongo_uri>')

GazeDB = client['GazeDB']  # Replace 'GazeDB' with your actual database name
ClickDB = client['ClickDB']
MetaDB = client['MetaDB']

INTUX = client['INTUX']
GazeDataCollection = INTUX['GazeData']
ClickDataCollection = INTUX['ClickData']
MetaDataCollection = INTUX['MetaData']

def normalizeRoll(rollNo):

    GazeRoll = GazeDB[rollNo]  # Replace 'CB.EN.U4CSE20401' with your actual collection name
    ClickRoll = ClickDB[rollNo]  # Replace 'YourClickCollection' with your actual collection name
    MetaRoll = MetaDB[rollNo]  # Replace 'YourMetaCollection' with your actual collection name

    # Find the minimum timestamp in the collection
    min_timestamp_result = GazeRoll.aggregate([
        {"$group": {"_id": None, "minTimestamp": {"$min": "$timestamp"}}}
    ])

    min_timestamp_result_list = list(min_timestamp_result)

    if not min_timestamp_result_list:
        raise Exception("Collection is empty",rollNo)

    min_timestamp = min_timestamp_result_list[0]["minTimestamp"]

    # Update each document with the normalizedTimestamp field
    GazeRoll.update_many({}, [
        {"$set": {"normalizedTimestamp": {"$subtract": ["$timestamp", min_timestamp]}}}
    ])

    ClickRoll.update_many({}, [
        {"$set": {"normalizedTimestamp": {"$subtract": ["$timestamp", min_timestamp]}}}
    ])

    metadata = MetaRoll.find_one({'rollNo': rollNo})
    if metadata:
        # Get screen dimensions
        screen_width = metadata['screenWidth']
        screen_height = metadata['screenHeight']

        # Update GazeData collection for the user
        GazeRoll.update_many(
            {},
            [
                {
                    '$set': {
                        'normalizedX': {'$divide': ['$x', screen_width]},
                        'normalizedY': {'$divide': ['$y', screen_height]}
                    }
                }
            ]
        )

        ClickRoll.update_many(
            {},
            [
                {
                    '$set': {
                        'normalizedX': {'$divide': ['$x', screen_width]},
                        'normalizedY': {'$divide': ['$y', screen_height]}
                    }
                }
            ]
        )

        GazeRoll.update_many({}, [{"$set": {"rollNo": rollNo}}])
        ClickRoll.update_many({}, [{"$set": {"rollNo": rollNo}}])

        # Insert documents into GazeData and ClickData collections in INTUX
        GazeDataCollection.insert_many(list(GazeRoll.find({})))
        ClickDataCollection.insert_many(list(ClickRoll.find({})))
        MetaDataCollection.insert_one(metadata)


def get_collection_names(database_name):
    client = MongoClient('<mongo_uri>')
    db = client[database_name]
    collection_names = db.list_collection_names()
    return collection_names

# Replace 'YourDatabaseName' with the actual name of your database
database_name = 'MetaDB'

# Get the collection names
collections = get_collection_names(database_name)

# Convert the list of collection names to a string
collections_string = list(collections)

for rollNo in collections_string:
    normalizeRoll(rollNo)
